### Introduction

In this project, I will classify images of birds by using transfer learning from a pre-trained network. I will use MobileNet V2 model developed at Google. I am beginner for computer vision and i am trying to learn more about it. I will follow transfer learning and fine-tuning which is tensorflow tutorial for the project as a guideline.

My aim is to complete an end to end computer vision problem with basic parts, such as bulding an input pipeline, compose the model, data augmentation etc.

Let's get started !!

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
print(tf.__version__)

### Data Processing

In [ ]:
PATH="../input/100-bird-species"
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'valid')
test_dir = os.path.join(PATH, 'test')

In [ ]:
# Create tf.data.Dataset for training, validation and test using Keras image_dataset_from_directory

BATCH_SIZE = 64
IMG_SIZE = (224,224)

train_dataset = image_dataset_from_directory(train_dir,
                                             shuffle=True,
                                             batch_size=BATCH_SIZE,
                                             image_size=IMG_SIZE)

validation_dataset = image_dataset_from_directory(validation_dir,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  image_size=IMG_SIZE)

test_dataset = image_dataset_from_directory(test_dir,
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            image_size=IMG_SIZE)

In [ ]:
# Control bathches sizes

print('Number of train batches: %d' % tf.data.experimental.cardinality(train_dataset))
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

In [ ]:
# The first nine images and labels from training set

class_names = train_dataset.class_names

plt.figure(figsize=(12,12))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
# for better data performance using buffered prefetching 

AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

### Simple Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
# Result for an image after apply the augmentation

for image, _ in train_dataset.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

### Create a base pre-trained model

In [ ]:
# Download mobilenet_v2 model architecture.

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# Create the base model from the pre-trained model MobileNet V2

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
# This feature extractor converts each 224x224x3 image into a 7x7x1280 block of features

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# Freeze the model weights before compile and train the model.

base_model.trainable = False

In [ ]:
# Convert the features to a single 1280 element vector per image

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
# Apply a tf.keras.layers.Dense layer to convert these features into 270 predictions per image

prediction_layer = tf.keras.layers.Dense(275, activation = 'softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
# Build a model by chaining together the data augmentation, rescaling, base_model and feature extractor layers using the Keras

inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
# Compile the model before training it. Since there are multi classes, I will use a Sparse Categorical Crossentropy loss 

base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['acc'])

In [ ]:
model.summary()

### Train The Model

In [ ]:
initial_epochs = 15
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset)

### Learning Curves

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,max(plt.ylim())])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

### Evaluation and prediction

In [ ]:
# Finaly we can verify the performance of the model on test set.

loss, accuracy = model.evaluate(test_dataset)
print('Test accuracy :', accuracy)

In [ ]:
# Plot prediction results

image_batch, label_batch = test_dataset.as_numpy_iterator().next()
print(image_batch)
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)

# plt.figure(figsize=(10, 10))
# for i in range(9):
#     ax = plt.subplot(3, 3, i + 1)
plt.imshow(image_batch[i].astype("uint8"))
plt.title(class_names[predicted_id[i]])
plt.axis("off")

In [ ]:
model.save('bird-96test')

In [ ]:
!zip -r 'bird-96test.zip' ./

In [ ]:
model = tf.keras.models.load_model('../input/birdmodel/bird-96test')

In [ ]:
def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224,3))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    ## Scaling
    x=x/255
    x = np.expand_dims(x, axis=0)
   

   

    preds = model.predict(x)
    preds=np.argmax(preds, axis=1)
   
    classes = {"0": "AFRICAN CROWNED CRANE", "1": "AFRICAN FIREFINCH", "2": "ALBATROSS", "3": "ALEXANDRINE PARAKEET", "4": "AMERICAN AVOCET", "5": "AMERICAN BITTERN", "6": "AMERICAN COOT", "7": "AMERICAN GOLDFINCH", "8": "AMERICAN KESTREL", "9": "AMERICAN PIPIT", "10": "AMERICAN REDSTART", "11": "ANHINGA", "12": "ANNAS HUMMINGBIRD", "13": "ANTBIRD", "14": "ARARIPE MANAKIN", "15": "ASIAN CRESTED IBIS", "16": "BALD EAGLE", "17": "BALI STARLING", "18": "BALTIMORE ORIOLE", "19": "BANANAQUIT", "20": "BANDED BROADBILL", "21": "BAR-TAILED GODWIT", "22": "BARN OWL", "23": "BARN SWALLOW", "24": "BARRED PUFFBIRD", "25": "BAY-BREASTED WARBLER", "26": "BEARDED BARBET", "27": "BEARDED REEDLING", "28": "BELTED KINGFISHER", "29": "BIRD OF PARADISE", "30": "BLACK & YELLOW bROADBILL", "31": "BLACK FRANCOLIN", "32": "BLACK SKIMMER", "33": "BLACK SWAN", "34": "BLACK TAIL CRAKE", "35": "BLACK THROATED BUSHTIT", "36": "BLACK THROATED WARBLER", "37": "BLACK VULTURE", "38": "BLACK-CAPPED CHICKADEE", "39": "BLACK-NECKED GREBE", "40": "BLACK-THROATED SPARROW", "41": "BLACKBURNIAM WARBLER", "42": "BLUE GROUSE", "43": "BLUE HERON", "44": "BOBOLINK", "45": "BORNEAN BRISTLEHEAD", "46": "BORNEAN LEAFBIRD", "47": "BROWN NOODY", "48": "BROWN THRASHER", "49": "BULWERS PHEASANT", "50": "CACTUS WREN", "51": "CALIFORNIA CONDOR", "52": "CALIFORNIA GULL", "53": "CALIFORNIA QUAIL", "54": "CANARY", "55": "CAPE MAY WARBLER", "56": "CAPUCHINBIRD", "57": "CARMINE BEE-EATER", "58": "CASPIAN TERN", "59": "CASSOWARY", "60": "CEDAR WAXWING", "61": "CHARA DE COLLAR", "62": "CHIPPING SPARROW", "63": "CHUKAR PARTRIDGE", "64": "CINNAMON TEAL", "65": "CLARKS NUTCRACKER", "66": "COCK OF THE  ROCK", "67": "COCKATOO", "68": "COMMON FIRECREST", "69": "COMMON GRACKLE", "70": "COMMON HOUSE MARTIN", "71": "COMMON LOON", "72": "COMMON POORWILL", "73": "COMMON STARLING", "74": "COUCHS KINGBIRD", "75": "CRESTED AUKLET", "76": "CRESTED CARACARA", "77": "CRESTED NUTHATCH", "78": "CROW", "79": "CROWNED PIGEON", "80": "CUBAN TODY", "81": "CURL CRESTED ARACURI", "82": "D-ARNAUDS BARBET", "83": "DARK EYED JUNCO", "84": "DOUBLE BARRED FINCH", "85": "DOWNY WOODPECKER", "86": "EASTERN BLUEBIRD", "87": "EASTERN MEADOWLARK", "88": "EASTERN ROSELLA", "89": "EASTERN TOWEE", "90": "ELEGANT TROGON", "91": "ELLIOTS  PHEASANT", "92": "EMPEROR PENGUIN", "93": "EMU", "94": "ENGGANO MYNA", "95": "EURASIAN GOLDEN ORIOLE", "96": "EURASIAN MAGPIE", "97": "EVENING GROSBEAK", "98": "FIRE TAILLED MYZORNIS", "99": "FLAME TANAGER", "100": "FLAMINGO", "101": "FRIGATE", "102": "GAMBELS QUAIL", "103": "GANG GANG COCKATOO", "104": "GILA WOODPECKER", "105": "GILDED FLICKER", "106": "GLOSSY IBIS", "107": "GO AWAY BIRD", "108": "GOLD WING WARBLER", "109": "GOLDEN CHEEKED WARBLER", "110": "GOLDEN CHLOROPHONIA", "111": "GOLDEN EAGLE", "112": "GOLDEN PHEASANT", "113": "GOLDEN PIPIT", "114": "GOULDIAN FINCH", "115": "GRAY CATBIRD", "116": "GRAY PARTRIDGE", "117": "GREAT POTOO", "118": "GREATOR SAGE GROUSE", "119": "GREEN JAY", "120": "GREEN MAGPIE", "121": "GREY PLOVER", "122": "GUINEA TURACO", "123": "GUINEAFOWL", "124": "GYRFALCON", "125": "HARPY EAGLE", "126": "HAWAIIAN GOOSE", "127": "HELMET VANGA", "128": "HIMALAYAN MONAL", "129": "HOATZIN", "130": "HOODED MERGANSER", "131": "HOOPOES", "132": "HORNBILL", "133": "HORNED GUAN", "134": "HORNED SUNGEM", "135": "HOUSE FINCH", "136": "HOUSE SPARROW", "137": "IMPERIAL SHAQ", "138": "INCA TERN", "139": "INDIAN BUSTARD", "140": "INDIAN PITTA", "141": "INDIGO BUNTING", "142": "JABIRU", "143": "JAVA SPARROW", "144": "KAKAPO", "145": "KILLDEAR", "146": "KING VULTURE", "147": "KIWI", "148": "KOOKABURRA", "149": "LARK BUNTING", "150": "LEARS MACAW", "151": "LILAC ROLLER", "152": "LONG-EARED OWL", "153": "MAGPIE GOOSE", "154": "MALABAR HORNBILL", "155": "MALACHITE KINGFISHER", "156": "MALEO", "157": "MALLARD DUCK", "158": "MANDRIN DUCK", "159": "MARABOU STORK", "160": "MASKED BOOBY", "161": "MASKED LAPWING", "162": "MIKADO  PHEASANT", "163": "MOURNING DOVE", "164": "MYNA", "165": "NICOBAR PIGEON", "166": "NOISY FRIARBIRD", "167": "NORTHERN BALD IBIS", "168": "NORTHERN CARDINAL", "169": "NORTHERN FLICKER", "170": "NORTHERN GANNET", "171": "NORTHERN GOSHAWK", "172": "NORTHERN JACANA", "173": "NORTHERN MOCKINGBIRD", "174": "NORTHERN PARULA", "175": "NORTHERN RED BISHOP", "176": "NORTHERN SHOVELER", "177": "OCELLATED TURKEY", "178": "OKINAWA RAIL", "179": "OSPREY", "180": "OSTRICH", "181": "OVENBIRD", "182": "OYSTER CATCHER", "183": "PAINTED BUNTIG", "184": "PALILA", "185": "PARADISE TANAGER", "186": "PARAKETT  AKULET", "187": "PARUS MAJOR", "188": "PEACOCK", "189": "PELICAN", "190": "PEREGRINE FALCON", "191": "PHILIPPINE EAGLE", "192": "PINK ROBIN", "193": "PUFFIN", "194": "PURPLE FINCH", "195": "PURPLE GALLINULE", "196": "PURPLE MARTIN", "197": "PURPLE SWAMPHEN", "198": "PYGMY KINGFISHER", "199": "QUETZAL", "200": "RAINBOW LORIKEET", "201": "RAZORBILL", "202": "RED BEARDED BEE EATER", "203": "RED BELLIED PITTA", "204": "RED BROWED FINCH", "205": "RED FACED CORMORANT", "206": "RED FACED WARBLER", "207": "RED HEADED DUCK", "208": "RED HEADED WOODPECKER", "209": "RED HONEY CREEPER", "210": "RED TAILED THRUSH", "211": "RED WINGED BLACKBIRD", "212": "RED WISKERED BULBUL", "213": "REGENT BOWERBIRD", "214": "RING-NECKED PHEASANT", "215": "ROADRUNNER", "216": "ROBIN", "217": "ROCK DOVE", "218": "ROSY FACED LOVEBIRD", "219": "ROUGH LEG BUZZARD", "220": "ROYAL FLYCATCHER", "221": "RUBY THROATED HUMMINGBIRD", "222": "RUFOUS KINGFISHER", "223": "RUFUOS MOTMOT", "224": "SAMATRAN THRUSH", "225": "SAND MARTIN", "226": "SCARLET IBIS", "227": "SCARLET MACAW", "228": "SHOEBILL", "229": "SHORT BILLED DOWITCHER", "230": "SMITHS LONGSPUR", "231": "SNOWY EGRET", "232": "SNOWY OWL", "233": "SORA", "234": "SPANGLED COTINGA", "235": "SPLENDID WREN", "236": "SPOON BILED SANDPIPER", "237": "SPOONBILL", "238": "SRI LANKA BLUE MAGPIE", "239": "STEAMER DUCK", "240": "STORK BILLED KINGFISHER", "241": "STRAWBERRY FINCH", "242": "STRIPPED SWALLOW", "243": "SUPERB STARLING", "244": "SWINHOES PHEASANT", "245": "TAIWAN MAGPIE", "246": "TAKAHE", "247": "TASMANIAN HEN", "248": "TEAL DUCK", "249": "TIT MOUSE", "250": "TOUCHAN", "251": "TOWNSENDS WARBLER", "252": "TREE SWALLOW", "253": "TRUMPTER SWAN", "254": "TURKEY VULTURE", "255": "TURQUOISE MOTMOT", "256": "UMBRELLA BIRD", "257": "VARIED THRUSH", "258": "VENEZUELIAN TROUPIAL", "259": "VERMILION FLYCATHER", "260": "VICTORIA CROWNED PIGEON", "261": "VIOLET GREEN SWALLOW", "262": "VULTURINE GUINEAFOWL", "263": "WATTLED CURASSOW", "264": "WHIMBREL", "265": "WHITE CHEEKED TURACO", "266": "WHITE NECKED RAVEN", "267": "WHITE TAILED TROPIC", "268": "WHITE THROATED BEE EATER", "269": "WILD TURKEY", "270": "WILSONS BIRD OF PARADISE", "271": "WOOD DUCK", "272": "YELLOW BELLIED FLOWERPECKER", "273": "YELLOW CACIQUE", "274": "YELLOW HEADED BLACKBIRD"}
    classes = {int(k):str(v) for k,v in classes.items()}
    
    return classes[int(preds)]

In [ ]:
model_predict('../input/100-bird-species/train/ALBATROSS/002.jpg', model)

In [ ]:

class_dict = {}
for i in range(0,len(class_names)):
    class_dict[i] = class_names[i]
    

In [ ]:
class_dict

In [ ]:
import json  
 
 
with open("classes.json", "w") as outfile: 
    json.dump(class_dict, outfile)

In [ ]:
load_model = tf.keras.models.load_model('../input/birdmodel/bird-96test')

In [ ]:
new_save = load_model.save('bird_model.h5')